<a href="https://colab.research.google.com/github/abhiramangit/Imag_reco/blob/master/Generic_FFNN_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm_notebook
from sklearn.metrics import log_loss,mean_squared_error,accuracy_score

In [0]:
data,labels = make_blobs(1000,2,centers=4,random_state=1)

In [0]:
plt.scatter(data[:,0],data[:,1],c = labels)

In [0]:
X_train,X_test,Y_train,Y_test = train_test_split(data,labels,stratify=labels,random_state=1)
print(X_train.shape,Y_train.shape)

In [0]:
## One Hot Encoding
enc = OneHotEncoder()

Y_OH_train = enc.fit_transform(np.expand_dims(Y_train,axis=1)).toarray()

In [0]:
class Generic_vectorized_FFNN:
  def __init__(self,X,layer_info,output_function='SoftMax'):
    self.W = {}
    self.B = {}

    self.nl = len(layer_info)
    self.layer_info = layer_info
    self.A = {}
    self.H = {}

    self.dA = {}
    self.dH = {}
    self.dW = {}
    self.dB = {}


    self.W[1] = np.random.randn(X.shape[1], self.layer_info[0])
    self.B[1] = np.random.rand(1, self.layer_info[0])
    self.A[0] = X
    self.H[0] = X

    for i in range(1,self.nl):
      self.W[i+1] = np.random.randn(self.layer_info[i-1], self.layer_info[i])
      self.B[i+1] = np.random.randn(1, self.layer_info[i])


  def sigmoid(self,X):
    return 1.0/(1.0+np.exp(-X))

  def softmax(self,X):
    exps = np.exp(X)
    summ = np.sum(exps,axis=1).reshape(-1,1)
    ret = exps/summ
    return ret

  def forward_pass(self,X):

    for i in range(self.nl):
      self.A[i+1] = np.matmul(self.A[i],self.W[i+1]) + self.B[i+1]   # (N,c) * (c,n) -> (N,n)
      if i==len( self.layer_info)-1:
        self.H[i+1] = self.softmax(self.A[i+1])
        return self.H[i+1]
      else:
        self.H[i+1] = self.sigmoid(self.A[i+1])     # (N,n)
  
  def grad_sigmoid(self,X):
    return X*(1-X)

  def grad_params(self,X,Y):
    self.forward_pass(X)

    self.dA[self.nl] = self.H[self.nl] - Y     # (N,2)
  
    ## Output Layer 
    for i in range(self.nl-1,-1,-1):
      self.dW[i+1] = np.matmul(self.H[i].T,self.dA[i+1])     # (2,N)*(N,n) -> (2,n)
      self.dB[i+1] = np.sum(self.dA[i+1],axis=0)

      
      if i!=0:
        self.dH[i] = np.matmul(self.dA[i+1],self.W[i+1].T) 
        self.dA[i] = np.multiply(self.dH[i],self.grad_sigmoid(self.H[i]))

  def fit(self,X,Y,epochs= 1, learning_rate= 1 ,display_loss = False):

    if display_loss == True :
      loss = {}
    
    for ind in tqdm_notebook(range(epochs),total = epochs, unit='epoch'):
      self.grad_params(X,Y)
      m = X.shape[0]
      for i in range(1,self.nl):
        self.W[i] -= learning_rate*(self.dW[i]/m)
        self.B[i] -= learning_rate*(self.dB[i]/m)


      if display_loss==True:
        Y_pred = self.predict(X)
        Y_val = np.argmax(Y,axis=1)
        Y_pred_val = np.argmax(Y_pred,axis=1)
        loss_val = mean_squared_error(Y_val,Y_pred_val)
        loss[ind] = loss_val

    loss_list = list(loss.values())

    if display_loss == True:
      
      plt.plot(loss_list,'*')
      plt.xlabel('epoch')
      plt.ylabel('mean loss')     
      plt.show()

  def predict(self, X):
    Y_pred = self.forward_pass(X)
    return np.array(Y_pred).squeeze()


        

In [0]:
obj = Generic_vectorized_FFNN(X_train,[2,3,4])
ret = obj.fit(X_train,Y_OH_train,epochs=4000,learning_rate=0.05,display_loss=True)


In [0]:
Y_pred_train = obj.predict(X_train)
Y_pred_vals_train = np.argmax(Y_pred_train,axis=1)

Y_train_val = np.argmax(Y_OH_train,axis=1)
#Y_pred_test = obj.predict(X_test)
#Y_pred_vals_test = np.argmax(Y_pred_test,axis=1)

train_accuracy = accuracy_score(Y_pred_vals_train,Y_train_val)
#test_accuracy = accuracy_score(Y_pred_vals_test,Y_test)

print("train_accuracy",train_accuracy)
#print("test accuracy",test_accuracy)

In [0]:
plt.scatter(X_train[:,0], X_train[:,1], c=Y_train, s=15*(np.abs(np.sign(Y_pred_vals_train-Y_train))+.1))
plt.show()